In [183]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

In [184]:
df_customers = pd.read_csv("data\olist_customers_dataset.csv")
print("df_customers", df_customers.shape)
df_sellers = pd.read_csv("data\olist_sellers_dataset.csv")
print("df_sellers", df_sellers.shape)
df_orders = pd.read_csv("data\olist_orders_dataset.csv")
print("df_orders", df_orders.shape)
df_products = pd.read_csv("data\olist_products_dataset.csv")
print("df_products", df_products.shape)
df_orderItems = pd.read_csv("data\olist_order_items_dataset.csv")
print("df_orderItems", df_orderItems.shape)
df_orderReviews = pd.read_csv("data\olist_order_reviews_dataset.csv")
print("df_orderReviews", df_orderReviews.shape)
df_orderPayments = pd.read_csv("data\olist_order_payments_dataset.csv")
print("df_orderPayments", df_orderPayments.shape)
df_geolocation = pd.read_csv("data\olist_geolocation_dataset.csv")
print("df_geolocation", df_geolocation.shape)
df_productCategory = pd.read_csv("data\product_category_name_translation.csv")
print("df_productCategory", df_productCategory.shape)


df_products_english = pd.merge(df_products,df_productCategory ,on="product_category_name",how="left")
print("df_products_english", df_products_english.shape)


df_customers (99441, 5)
df_sellers (3095, 4)
df_orders (99441, 8)
df_products (32951, 9)
df_orderItems (112650, 7)
df_orderReviews (100000, 7)
df_orderPayments (103886, 5)
df_geolocation (1000163, 5)
df_productCategory (71, 2)
df_products_english (32951, 10)


In [185]:
def getDateTime(date_str):
    return datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')


In [186]:
def Merge_Orders_and_Customers():
    return pd.merge(df_orders,df_customers,on="customer_id",how="left")
    

In [187]:
def GetCustomerLatestOrderDate(df_orders_customers,df_customers):
    df_customers['latest_order_date'] = df_orders_customers.groupby("customer_id")["order_purchase_timestamp"].transform(max)
    df_customers['latest_order_date'] = pd.to_datetime(df_customers['latest_order_date'])
    return df_customers

In [188]:
def SetChurnCustomers(df_customers):
    # lets take the latest order date (2018-10-17 17:30:18) as the current date and 
    # call user a churn user if he/she hasnt placed an order for last 3 months
    # churn_user = True when user hasnt ordered for last 3 months
    se_date = GetMaxColumnValue(df_customers,"latest_order_date")
    
    df_customers["churn_user"] = True
    df_customers.loc[ df_customers['latest_order_date'] >= ( se_date - timedelta( days = (3*30.42))), "churn_user"] = False
    
    return df_customers

In [189]:
def GetRowsWhereColumnValueEquals(df,columnName,value):
    return df.loc[df[columnName] == value]

def GetMaxColumnValue(df,columnName):
    return df[columnName].max()

def GetMinColumnValue(df,columnName):
    return df[columnName].min()

def GetUnqiueColumnValues(df,columnName):
    return df[columnName].unique()

def GetUnqiueColumnValuesLength(df,columnName):
    return len(df[columnName].unique())

def GetCountOfGroupBy(df,GroupbyOn,ColumnCountOn):
    return df.groupby(GroupbyOn)[ColumnCountOn].count()

In [190]:
df_orders_customers =  Merge_Orders_and_Customers()
df_customers = GetCustomerLatestOrderDate(df_orders_customers,df_customers)
df_customers = SetChurnCustomers(df_customers)

In [125]:
NumberOfCustomersInCity = GetCountOfGroupBy(df_customers,"customer_city","customer_id")
MaxCustomerCity = NumberOfCustomersInCity.idxmax()
MaxCustomerCityValue = NumberOfCustomersInCity.max()

MaxCustomerCity,MaxCustomerCityValue

('sao paulo', 15540)

In [149]:
x = df_orders.groupby("customer_id").count()
print(x["order_id"].max())
# or
print(df_orders["customer_id"].nunique())
print(df_orders.shape)

# that means no customer has more than 1 order

1
99441
(99441, 8)


In [182]:
# try uncommenting both and see the results
# df_orderPayments.groupby("payment_sequential").count()
GetRowsWhereColumnValueEquals(df_orderPayments,"payment_sequential",1)  .head(10)


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
5,298fcdf1f73eb413e4d26d01b25bc1cd,1,credit_card,2,96.12
6,771ee386b001f06208a7419e4fc1bbd7,1,credit_card,1,81.16
7,3d7239c394a212faae122962df514ac7,1,credit_card,3,51.84
8,1f78449c87a54faf9e96e88ba1491fa9,1,credit_card,6,341.09
9,0573b5e23cbd798006520e1d5b4c6714,1,boleto,1,51.95


112.8